In [1380]:
# loading package
import numpy as np
import pandas as pd
import igraph
from scipy.optimize import minimize
from scipy.special import digamma
import networkx as nx

## Loading data

Set the initial parameters for $\alpha, \beta, \delta, \gamma, \lambda$

In [1381]:
np.random.seed(1)

p = 5
E_true  = np.zeros((p, p), dtype=int)

for j in range(1, p):
    E_true[j, j - 1] = 1

alpha_true = np.zeros((p, p))
alpha_true[E_true == 1] = np.random.uniform(0.5, 2, size=np.sum(E_true))
beta_true = np.zeros((p, p))
beta_true[E_true == 1] = np.random.uniform(-2, -0.5, size=np.sum(E_true))
delta_true = np.random.uniform(-1.5, -1, size=p)
gamma_true = np.random.uniform(1, 1.5, size=p)
lambda_true = np.exp(np.random.uniform(-2, 2, size=p))


# Generate synthetic data from the specified linear ZiG-DAG
n = 50  # Sample size
dat = np.zeros((n, p))
#order_nodes = list(igraph.topological_sort(igraph.Graph.Erdos_Renyi(n=p, p=0.5, directed=True)))
order_nodes = np.arange(0,5)

for j in order_nodes:
    pi_true = np.exp(np.dot(dat, alpha_true[j, :]) + delta_true[j])
    pi_true = pi_true / (1 + pi_true)
    pi_true[np.isnan(pi_true)] = 1
    mu_true = np.exp(np.dot(dat, beta_true[j, :]) + gamma_true[j])
    # use r output at first for following data
    #dat[:, j] = (1 - np.random.binomial(1, pi_true, size=n)) * rhP(n, lambda_true[j], mu_true)

dat = pd.read_csv('syn_data.csv',sep=',')
dat = np.array(dat)


In [1384]:
#dat = pd.read_csv('/Users/a080528/Desktop/Purdue/Prof. Ju/Research/syn_data_n250.csv',sep=',')
#dat = np.array(dat)

**Linear ZiGDAG**

In [1385]:
def f11_cpp(z, a, b, Iter, tol):
    
    fac = 1
    temp = fac
    series = 0
    
    for e in range(Iter):
        fac *= (a/b) * (z/(e+1))

        if np.isnan(fac): fac = 0
        series = temp + fac
        
        if (not np.isfinite(series) or np.abs(series - temp) < tol): 
            return series
        
        temp = series
        
        a += 1
        b += 1
        
    return series

def f11_for_each(z, a, b, Iter, tol):
    
    '''
    
    z: vector Mu_vector is 2D vector
    x: 1
    y: lambda
    
    '''
    
    num = len(z)
    val = np.empty((num, 1))

    for i in range(num):
        val[i, 0] = f11_cpp(z[i, 0], a, b, Iter, tol)
        
    return val

def log_ascfacto(z, n):
    p = len(n)
    out = np.zeros((p, 1))
    
    for i in range(p):
        for j in range(n[i]):
            out[i] += np.log(z+j)
            
    return out
    
    
def dZIHP_cpp(z, y, x, lower, upper):

    z = z.reshape(1,-1)

    pn, pp = x.shape
    x1 = np.concatenate([x, np.ones((n,1))], axis = 1)

    Pi = np.exp(x1 @ z[:,0:pp+1].T)
    Pi = Pi/(1+Pi)
    Pi[~np.isfinite(Pi)] = 1
    
    Mu = np.exp(x1 @ z[:,pp+1:2*pp+2].T)
    Lambda = np.exp(z[:,-1])
    
    Y0 = np.where(y<=1e-8)
    Y1 = np.where(y>1e-8)
    eval_F11 = f11_for_each(Mu, 1, Lambda, 10000, 1e-8)
    
    #return Y0, Y1, eval_F11, Pi, Mu
    # keep working on likelihood function
    llik = np.sum(np.log(Pi[Y0] + (1-Pi[Y0])/eval_F11[Y0]))+np.sum(np.log(1-Pi[Y1])-log_ascfacto(Lambda, y[Y1])-np.log(eval_F11[Y1]) + (y[Y1].reshape(-1,1) % np.log(Mu[Y1])))

    if ((not np.isfinite(llik)) and (llik<0)):
        return lower
    elif ((not np.isfinite(llik)) and (llik>0)):
        return upper
    else:
        return llik

def gradF11b_cpp(z, b, Iter, tol):
    con = digamma(b) * f11_cpp(z, 1, b, Iter, tol)
    fac = digamma(b)
    temp = fac
    series = 0
    
    for i in range(Iter):
        fac *= (digamma(b+1)/digamma(b)) * (z/b)
        
        if np.isnan(fac): fac = 0
        series = temp + fac
        
        if (not np.isfinite(series)) or (np.abs(series - temp) < tol):
            return (con - series)
        
        temp = series
        b += 1
        
    return (con-series)

def gradF11b_for_each(z, b, Iter, tol):
    
    '''
    
    z: vector Mu_vector is 2D vector
    x: 1
    y: lambda
    
    '''
    
    num = len(z)
    val = np.empty((num, 1))

    for i in range(num):
        val[i, 0] = gradF11b_cpp(z[i, 0], b, Iter, tol)
        
    return val


def gradZIHP_cpp(z, y, x):
    
    z = z.reshape(1,-1)

    pn, pp = x.shape
    x1 = np.concatenate([x, np.ones((n,1))], axis = 1)


    #llmd = z[:,2*xp+2]

    Pi = np.exp(x1 @ z[:,0:pp+1].T)
    Pi = Pi/(1+Pi)
    Pi[~np.isfinite(Pi)] = 1

    Mu = np.exp(x1 @ z[:,pp+1:2*pp+2].T)
    Lambda = np.exp(z[:,-1])

    Y0 = np.where(y<=1e-8)
    Y1 = np.where(y>1e-8)
    X10 = x1[Y0];
    X11 = x1[Y1];
    
    eval1_F11 = f11_for_each(Mu, 1, Lambda, 10000, 1e-8)
    eval2_F11 = f11_for_each(Mu, 2, Lambda+1, 10000, 1e-8)
    eval_gradF11b = gradF11b_for_each(Mu, Lambda, 10000, 1e-8)
    dZIHP0 = Pi[Y0] + (1-Pi[Y0])/eval1_F11[Y0]
    
    # check p1, p2, and p3
    
    grad = np.zeros(2 * pp + 3)
    # C++ code for p1
    #arma::sum(X10.each_col() % ((1 - Pi.elem(Y0)) % (Pi.elem(Y0) - Pi.elem(Y0) / eval1_F11.elem(Y0)) / dZIHP0), 0) - arma::sum(X11.each_col() % Pi.elem(Y1), 0);
    p1 = np.sum(X10 % ((1 - Pi[Y0]) % ((Pi[Y0] - (Pi[Y0]/ eval1_F11[Y0])) / dZIHP0)), axis = 0) - np.sum(X11 % Pi[Y1], axis =0) 
    
    # C++ code for p2
    # -arma::sum(X10.each_col() % (Mu.elem(Y0) % (1 - Pi.elem(Y0)) / Lambda % eval2_F11.elem(Y0) / arma::square(eval1_F11.elem(Y0)) / dZIHP0), 0) -
    # arma::sum(X11.each_col() % (Mu.elem(Y1) / Lambda % eval2_F11.elem(Y1) / eval1_F11.elem(Y1) - y.elem(Y1)), 0);
    p2 = -np.sum(X10 % Mu[Y0] % ((1-Pi[Y0])/Lambda) % (eval2_F11[Y0]/np.square(eval1_F11[Y0])/dZIHP0), axis = 0) -np.sum(X11 % (((Mu[Y1]/Lambda) % (eval2_F11[Y1]/eval1_F11[Y1])) -y[Y1].reshape(-1,1)), axis = 0)

    # C++ code for p3
    # -arma::accu((1 - Pi.elem(Y0)) % eval_gradF11b.elem(Y0) / arma::square(eval1_F11.elem(Y0)) / dZIHP0) - 
    # arma::accu(eval_gradF11b.elem(Y1) / eval1_F11.elem(Y1) + digamma_arma(Lambda + y.elem(Y1)) - R::digamma(Lambda))) * Lambda;
    p3 = (-np.sum((1-Pi[Y0]) % (eval_gradF11b[Y0]/np.square(eval1_F11[Y0])/dZIHP0), axis = 0) -np.sum((eval_gradF11b[Y1]/eval1_F11[Y1] + digamma(Lambda + y[Y1].reshape(-1,1)) - digamma(Lambda)), axis = 0)) * Lambda
    
    grad[0:pp+1] = p1
    grad[pp+1: 2*pp+2] = p2
    grad[2*pp+2] = p3

    return grad

def create_dag(M):
    G = nx.DiGraph()
    snode = np.where(E_cand == 1)[0]
    enode = np.where(E_cand == 1)[1]
    
    for i in range(len(snode)):
        G.add_edge(snode[i], enode[i])
    
    return G

def initialpars(ps):
    start_pars = []
    
    for e in ps:
        if type(e) == np.ndarray:
            start_pars.extend(e)
        elif type(e) == np.float64:
            start_pars.append(e)

    return (np.array(start_pars))

## Testing

In [838]:
testE = est_curr['E'].copy()
testE[0, 1] = 1
testE

array([[0., 1., 0., 0., 0.],
       [0., 0., 0., 0., 0.],
       [0., 0., 0., 0., 0.],
       [0., 0., 0., 0., 0.],
       [0., 0., 0., 0., 0.]])

In [1405]:
def dZIHP_cpp(z, y, x, lower, upper):

    z = z.reshape(1,-1)

    pn, pp = x.shape
    x1 = np.concatenate([x, np.ones((n,1))], axis = 1)

    Pi = np.exp(x1 @ z[:,0:pp+1].T)
    Pi = Pi/(1+Pi)
    Pi[~np.isfinite(Pi)] = 1
    
    Mu = np.exp(x1 @ z[:,pp+1:2*pp+2].T)
    Lambda = np.exp(z[:,-1])
    
    Y0 = np.where(y<=1e-8)
    Y1 = np.where(y>1e-8)
    eval_F11 = f11_for_each(Mu, 1, Lambda, 10000, 1e-8)
    
    #return Y0, Y1, eval_F11, Pi, Mu
    # keep working on likelihood function
    llik = np.sum(np.log(Pi[Y0] + (1-Pi[Y0])/eval_F11[Y0]))+np.sum(np.log(1-Pi[Y1])-log_ascfacto(Lambda, y[Y1])-np.log(eval_F11[Y1]) + (y[Y1].reshape(-1,1) % np.log(Mu[Y1])))

    if ((not np.isfinite(llik)) and (llik<0)):
        return lower
    elif ((not np.isfinite(llik)) and (llik>0)):
        return upper
    else:
        return llik
    
def gradZIHP_cpp(z, y, x):
    
    z = z.reshape(1,-1)

    pn, pp = x.shape
    x1 = np.concatenate([x, np.ones((n,1))], axis = 1)

    Pi = np.exp(x1 @ z[:,0:pp+1].T)
    Pi = Pi/(1+Pi)
    Pi[~np.isfinite(Pi)] = 1

    Mu = np.exp(x1 @ z[:,pp+1:2*pp+2].T)
    Lambda = np.exp(z[:,-1])

    Y0 = np.where(y<=1e-8)
    Y1 = np.where(y>1e-8)
    X10 = x1[Y0]
    X11 = x1[Y1]
    
    eval1_F11 = f11_for_each(Mu, 1, Lambda, 10000, 1e-8)
    eval2_F11 = f11_for_each(Mu, 2, Lambda+1, 10000, 1e-8)
    print([eval1_F11, eval2_F11])
    eval_gradF11b = gradF11b_for_each(Mu, Lambda, 10000, 1e-8)
    dZIHP0 = Pi[Y0] + (1-Pi[Y0])/eval1_F11[Y0]
    
    grad = np.zeros(2 * pp + 3)
    # C++ code for p1
    #arma::sum(X10.each_col() % ((1 - Pi.elem(Y0)) % (Pi.elem(Y0) - Pi.elem(Y0) / eval1_F11.elem(Y0)) / dZIHP0), 0) - arma::sum(X11.each_col() % Pi.elem(Y1), 0);
    p1 = np.sum(X10 % ((1 - Pi[Y0]) % ((Pi[Y0] - (Pi[Y0]/ eval1_F11[Y0])) / dZIHP0)), axis = 0) - np.sum(X11 % Pi[Y1], axis =0) 
    
    # C++ code for p2
    # -arma::sum(X10.each_col() % (Mu.elem(Y0) % (1 - Pi.elem(Y0)) / Lambda % eval2_F11.elem(Y0) / arma::square(eval1_F11.elem(Y0)) / dZIHP0), 0) -
    # arma::sum(X11.each_col() % (Mu.elem(Y1) / Lambda % eval2_F11.elem(Y1) / eval1_F11.elem(Y1) - y.elem(Y1)), 0);
    p2 = -np.sum(X10 % Mu[Y0] % ((1-Pi[Y0])/Lambda) % (eval2_F11[Y0]/np.square(eval1_F11[Y0])/dZIHP0), axis = 0) -np.sum(X11 % (((Mu[Y1]/Lambda) % (eval2_F11[Y1]/eval1_F11[Y1])) -y[Y1].reshape(-1,1)), axis = 0)

    # C++ code for p3
    # -arma::accu((1 - Pi.elem(Y0)) % eval_gradF11b.elem(Y0) / arma::square(eval1_F11.elem(Y0)) / dZIHP0) - 
    # arma::accu(eval_gradF11b.elem(Y1) / eval1_F11.elem(Y1) + digamma_arma(Lambda + y.elem(Y1)) - R::digamma(Lambda))) * Lambda;
    p3 = (-np.sum((1-Pi[Y0]) % (eval_gradF11b[Y0]/np.square(eval1_F11[Y0])/dZIHP0), axis = 0) -np.sum((eval_gradF11b[Y1]/eval1_F11[Y1] + digamma(Lambda + y[Y1].reshape(-1,1)) - digamma(Lambda)), axis = 0)) * Lambda
    
    grad[0:pp+1] = p1
    grad[pp+1: 2*pp+2] = p2
    grad[2*pp+2] = p3

    return grad


In [1400]:
j = 3
pa_j = np.array([True, False, False, False, False])
p_j = sum(pa_j)

if np.array(est_curr["alpha"][j, pa_j]).size == 0 and np.array(est_curr["beta"][j, pa_j]).size == 0:
    start_j = [est_curr["delta"][j], est_curr["gamma"][j],
                np.log(est_curr["lambda"][j])]
else:
    start_j = [(est_curr["alpha"][j, pa_j]), est_curr["delta"][j], est_curr["beta"][j, pa_j], est_curr["gamma"][j],
                np.log(est_curr["lambda"][j])]
    
start_j = initialpars(start_j)
pars = np.array(start_j)

In [1401]:
pars

array([0., 0., 0., 1., 0.])

In [1406]:
gradZIHP_cpp(pars, dat[:, j], dat[:, pa_j])

[array([[15.15426224],
       [ 1.16538175],
       [ 1.1114581 ],
       [ 1.0866647 ],
       [ 1.07087962],
       [ 1.06130321],
       [ 1.05400422],
       [ 1.04825715],
       [ 1.04432562],
       [ 1.04098608],
       [ 1.03811426],
       [ 1.03561837],
       [ 1.03342914],
       [ 1.03149337],
       [ 1.02976944],
       [ 1.0282244 ],
       [ 1.02683178],
       [ 1.0255701 ],
       [ 1.02464307],
       [ 1.02378089],
       [ 1.02297699],
       [ 1.02222566],
       [ 1.0215219 ],
       [ 1.02086133],
       [ 1.0202401 ],
       [ 1.01965479],
       [ 1.01910238],
       [ 1.01858017],
       [ 1.01808575],
       [ 1.01761695],
       [ 1.01717184],
       [ 1.01674867],
       [ 1.01634585],
       [ 1.01596195],
       [ 1.01559567],
       [ 1.01524582],
       [ 1.01491132],
       [ 1.01459118],
       [ 1.0142845 ],
       [ 1.01399044],
       [ 1.01370825],
       [ 1.01343721],
       [ 1.01317668],
       [ 1.01292607],
       [ 1.0126848 ],
       [ 

array([ 9.43754379e-02,  8.32839592e-02,  5.08128658e+01,  1.05835785e+02,
       -1.08685381e+05])

In [1388]:
dZIHP_cpp(pars, dat[:, j], dat[:, pa_j], -np.finfo(float).max, np.finfo(float).max)

-326.5069340675871

In [1389]:
#minimize(fun= lambda z : dZIHP_cpp(z, dat[:, j], dat[:, pa_j], -np.finfo(float).max, np.finfo(float).max)
         #,x0=pars, jac=lambda z: gradZIHP_cpp(z, dat[:, j], dat[:, pa_j]), method='BFGS')

      fun: -326.5069340675871
 hess_inv: array([[1, 0, 0, 0, 0],
       [0, 1, 0, 0, 0],
       [0, 0, 1, 0, 0],
       [0, 0, 0, 1, 0],
       [0, 0, 0, 0, 1]])
      jac: array([ 6.33715142e-02,  8.82266820e-02,  4.08562882e+01,  4.38970679e+01,
       -5.27248020e+04])
  message: 'Desired error not necessarily achieved due to precision loss.'
     nfev: 86
      nit: 0
     njev: 74
   status: 2
  success: False
        x: array([0., 0., 0., 1., 0.])

In [1392]:
np.sum(bic_iter)

4477.390920050204

In [1391]:
bic_iter

array([1676.23749341,  312.62456024,  260.39365112, 1563.38527813,
        664.74993715])

In [1390]:
n, p = dat.shape
starting_dag = np.zeros((p, p))

bic_curr = np.empty(p)
est_curr = {
    "E": starting_dag,
    "alpha": np.zeros((p, p)),
    "beta": np.zeros((p, p)),
    "delta": np.zeros(p),
    #"gamma": np.zeros(p),
    "gamma": np.ones(p),
    "lambda": np.ones(p),
}
maxiter = 500
tol = 1e-12

for j in range(p):
    pa_j = (est_curr['E'][j, :] == 1)
    
    if np.array(est_curr["alpha"][j, pa_j]).size == 0 and np.array(est_curr["beta"][j, pa_j]).size == 0:
        start_j = [est_curr["delta"][j], est_curr["gamma"][j],
                    np.log(est_curr["lambda"][j])]
    else:
        start_j = [(est_curr["alpha"][j, pa_j]), est_curr["delta"][j], est_curr["beta"][j, pa_j], est_curr["gamma"][j],
                    np.log(est_curr["lambda"][j])]
    
    start_j = initialpars(start_j)
    pars = np.array(start_j)
    
    out_j = minimize(fun= lambda z : dZIHP_cpp(z, dat[:, j], dat[:, pa_j], -np.finfo(float).max, np.finfo(float).max)
         , x0=pars, jac=lambda z: gradZIHP_cpp(z, dat[:, j], dat[:, pa_j]), method='BFGS')
    
    p_j = sum(pa_j)
    bic_curr[j] = -2 * out_j.fun + (2 * p_j + 3) * np.log(n)

    if p_j > 0:
        
        est_curr["alpha"][j, pa_j] = out_j.x[0 : p_j]
        est_curr["beta"][j, pa_j]  = out_j.x[(p_j + 1) : (2 * p_j + 1)]

    est_curr["delta"][j]  = out_j.x[p_j]
    est_curr["gamma"][j]  = out_j.x[2 * p_j + 1]
    est_curr["lambda"][j] = np.exp(out_j.x[2 * p_j + 2])
    
    bic_iter = bic_curr.copy()
    est_iter = est_curr.copy()

In [1350]:
def dZIHP_cpp(z, y, x, lower, upper):

    z = z.reshape(1,-1)

    pn, pp = x.shape
    x1 = np.concatenate([x, np.ones((n,1))], axis = 1)

    Pi = np.exp(x1 @ z[:,0:pp+1].T)
    Pi = Pi/(1+Pi)
    Pi[~np.isfinite(Pi)] = 1
    
    Mu = np.exp(x1 @ z[:,pp+1:2*pp+2].T)
    Lambda = np.exp(z[:,-1])
    
    Y0 = np.where(y<=1e-8)
    Y1 = np.where(y>1e-8)
    eval_F11 = f11_for_each(Mu, 1, Lambda, 10000, 1e-8)
    
    
    #return Y0, Y1, eval_F11, Pi, Mu
    # keep working on likelihood function
    llik = np.sum(np.log(Pi[Y0] + (1-Pi[Y0])/eval_F11[Y0]))+np.sum(np.log(1-Pi[Y1])-log_ascfacto(Lambda, y[Y1])-np.log(eval_F11[Y1]) + (y[Y1].reshape(-1,1) % np.log(Mu[Y1])))
    print(llik)
    if ((not np.isfinite(llik)) and (llik<0)):
        return lower
    elif ((not np.isfinite(llik)) and (llik>0)):
        return upper
    else:
        return llik

In [1368]:
j = 3
pa_j = np.array([True, False, False, False, False])
p_j = sum(pa_j)

if np.array(est_curr["alpha"][j, pa_j]).size == 0 and np.array(est_curr["beta"][j, pa_j]).size == 0:
    start_j = [est_curr["delta"][j], est_curr["gamma"][j],
                np.log(est_curr["lambda"][j])]
else:
    start_j = [(est_curr["alpha"][j, pa_j]), est_curr["delta"][j], est_curr["beta"][j, pa_j], est_curr["gamma"][j],
                np.log(est_curr["lambda"][j])]
    
start_j = initialpars(start_j)
pars = np.array(start_j)
    
#out_j = minimize(fun= lambda z : dZIHP_cpp(z, dat[:, j], dat[:, pa_j], -np.finfo(float).max, np.finfo(float).max)
         #, x0=pars, jac=lambda z: gradZIHP_cpp(z, dat[:, j], dat[:, pa_j]), method='BFGS')

#-2 * out_j.fun + (2 * p_j + 3) * np.log(n)

In [1328]:
maxiter = 1
for Iter in range(maxiter):
    IMPROV = False
    for j in range(p):
        for k in range(p):
            print(j, k)
            if j == k:
                continue

            if est_curr["E"][j, k] == 0:
                E_cand = est_curr["E"].copy()
                E_cand[j, k] = 1

                # create dag for E_cand
                G_cand = create_dag(E_cand)
                if not nx.is_directed_acyclic_graph(G_cand): 
                    continue

                pa_j = (E_cand[j, ] == 1)
                if np.array(est_curr["alpha"][j, pa_j]).size == 0 and np.array(est_curr["beta"][j, pa_j]).size == 0:
                    start_j = [est_curr["delta"][j], est_curr["gamma"][j],
                                np.log(est_curr["lambda"][j])]
                else:
                    start_j = [(est_curr["alpha"][j, pa_j]), est_curr["delta"][j], est_curr["beta"][j, pa_j], est_curr["gamma"][j],
                                np.log(est_curr["lambda"][j])]

                start_j = initialpars(start_j)
                pars = np.array(start_j)

                out_j = minimize(fun= lambda z : dZIHP_cpp(z, dat[:, j], dat[:, pa_j], -np.finfo(float).max, np.finfo(float).max)
                     , x0=pars, jac=lambda z: gradZIHP_cpp(z, dat[:, j], dat[:, pa_j]), method='BFGS')

                p_j = sum(pa_j)
                
                bic_cand = bic_curr.copy()
                bic_cand[j] = -2 * out_j.fun + (2 * p_j + 3) * np.log(n)
                
                print(np.sum(bic_cand), np.sum(bic_iter))
                if np.isfinite(np.sum(bic_cand)) and np.sum(bic_cand) < np.sum(bic_iter) - tol:
                    IMPROV = True
                    bic_iter = bic_cand
                    est_iter = est_curr.copy()
                    est_iter["E"] = E_cand
                    est_iter["alpha"][j, pa_j] = out_j.x[:p_j]
                    est_iter["beta"][j, pa_j] = out_j.x[(p_j + 1) : (2 * p_j + 1)]
                    est_iter["delta"][j] = out_j.x[p_j]
                    est_iter["gamma"][j] = out_j.x[2 * p_j + 1]
                    est_iter["lambda"][j] = np.exp(out_j.x[2 * p_j + 2])
            else:
                E_cand = est_curr["E"].copy()
                E_cand[j, k] = 1

                pa_j = (E_cand[j, ] == 1)
                if np.array(est_curr["alpha"][j, pa_j]).size == 0 and np.array(est_curr["beta"][j, pa_j]).size == 0:
                    start_j = [est_curr["delta"][j], est_curr["gamma"][j],
                                np.log(est_curr["lambda"][j])]
                else:
                    start_j = [(est_curr["alpha"][j, pa_j]), est_curr["delta"][j], est_curr["beta"][j, pa_j], est_curr["gamma"][j],
                                np.log(est_curr["lambda"][j])]

                start_j = initialpars(start_j)
                pars = np.array(start_j)

                out_j = minimize(fun= lambda z : dZIHP_cpp(z, dat[:, j], dat[:, pa_j], -np.finfo(float).max, np.finfo(float).max)
                     , x0=pars, jac=lambda z: gradZIHP_cpp(z, dat[:, j], dat[:, pa_j]), method='BFGS')

                p_j = sum(pa_j)
                bic_cand = bic_curr
                bic_cand[j] = -2 * out_j.fun + (2 * p_j + 3) * np.log(n)
                
                    
                if np.isfinite(np.sum(bic_cand)) and np.sum(bic_cand) < np.sum(bic_iter) - tol:
                    
                    IMPROV = True
                    bic_iter = bic_cand.copy()
                    est_iter = est_curr.copy()
                    est_iter["E"] = E_cand
                    est_iter["alpha"][j, pa_j] = out_j.x[:p_j]
                    est_iter["beta"][j, pa_j] = out_j.x[(p_j + 1) : (2 * p_j + 1)]
                    est_iter["delta"][j] = out_j.x[p_j]
                    est_iter["gamma"][j] = out_j.x[2 * p_j + 1]
                    est_iter["lambda"][j] = np.exp(out_j.x[2 * p_j + 2])
            
            

0 0
0 1
1302.8179346595352 1279.3457966269664
0 2
1302.8179346595352 1279.3457966269664
0 3
1302.8179346595352 1279.3457966269664
0 4
1302.8179346595352 1279.3457966269664
1 0
1302.8179346595352 1279.3457966269664
1 1
1 2
1302.8179346595352 1279.3457966269664
1 3
1302.8179346595352 1279.3457966269664
1 4
1302.8179346595352 1279.3457966269664
2 0
1302.8179346595352 1279.3457966269664
2 1
1302.8179346595352 1279.3457966269664
2 2
2 3
1302.8179346595352 1279.3457966269664
2 4
1302.8179346595352 1279.3457966269664
3 0


/var/folders/2_/x3tlt3rs6j1ftvcyhmfh124c0000gn/T/ipykernel_15069/426753197.py:21: RuntimeWarning: divide by zero encountered in log
  llik = np.sum(np.log(Pi[Y0] + (1-Pi[Y0])/eval_F11[Y0]))+np.sum(np.log(1-Pi[Y1])-log_ascfacto(Lambda, y[Y1])-np.log(eval_F11[Y1]) + (y[Y1].reshape(-1,1) % np.log(Mu[Y1])))
/var/folders/2_/x3tlt3rs6j1ftvcyhmfh124c0000gn/T/ipykernel_15069/426753197.py:13: RuntimeWarning: overflow encountered in exp
  Lambda = np.exp(z[:,-1])
/var/folders/2_/x3tlt3rs6j1ftvcyhmfh124c0000gn/T/ipykernel_15069/4254713221.py:141: RuntimeWarning: overflow encountered in exp
  Lambda = np.exp(z[:,-1])
/var/folders/2_/x3tlt3rs6j1ftvcyhmfh124c0000gn/T/ipykernel_15069/4254713221.py:94: RuntimeWarning: invalid value encountered in true_divide
  fac *= (digamma(b+1)/digamma(b)) * (z/b)


KeyboardInterrupt: 

In [1326]:
print(bic_iter)

[427.00937419 123.61234531 111.45424237 413.20186583 204.06796893]


## Testing (Whole program)

In [1154]:
n, p = dat.shape
starting_dag = np.zeros((p, p))

bic_curr = np.empty(p)
est_curr = {
    "E": starting_dag,
    "alpha": np.zeros((p, p)),
    "beta": np.zeros((p, p)),
    "delta": np.zeros(p),
    #"gamma": np.zeros(p),
    "gamma": np.ones(p),
    "lambda": np.ones(p),
}
maxiter = 500
tol = 1e-12

for j in range(p):

    pa_j = (est_curr['E'][j, :] == 1)
    
    if np.array(est_curr["alpha"][j, pa_j]).size == 0 and np.array(est_curr["beta"][j, pa_j]).size == 0:
        start_j = [est_curr["delta"][j], est_curr["gamma"][j],
                    np.log(est_curr["lambda"][j])]
    else:
        start_j = [(est_curr["alpha"][j, pa_j]), est_curr["delta"][j], est_curr["beta"][j, pa_j], est_curr["gamma"][j],
                    np.log(est_curr["lambda"][j])]
    
    start_j = initialpars(start_j)
    pars = np.array(start_j, dtype=object)
    
    out_j = minimize(fun= lambda z : dZIHP_cpp(z, dat[:, j], dat[:, pa_j], -np.finfo(float).max, np.finfo(float).max)
         , x0=pars, jac=lambda z: gradZIHP_cpp(z, dat[:, j], dat[:, pa_j]), method='BFGS')
    
    p_j = sum(pa_j)
    bic_curr[j] = -2 * out_j.fun + (2 * p_j + 3) * np.log(n)

    if p_j > 0:
        
        est_curr["alpha"][j, pa_j] = out_j.x[0 : p_j]
        est_curr["beta"][j, pa_j]  = out_j.x[(p_j + 1) : (2 * p_j + 1)]

    est_curr["delta"][j]  = out_j.x[p_j]
    est_curr["gamma"][j]  = out_j.x[2 * p_j + 1]
    est_curr["lambda"][j] = np.exp(out_j.x[2 * p_j + 2])
    
    bic_iter = bic_curr
    est_iter = est_curr
    
for Iter in range(maxiter):
    IMPROV = False
    for j in range(p):
        for k in range(p):
            if j == k:
                continue

            if est_curr["E"][j, k] == 0:
                E_cand = est_curr["E"].copy()
                E_cand[j, k] = 1

                # create dag for E_cand
                G_cand = create_dag(E_cand)
                if not nx.is_directed_acyclic_graph(G_cand): 
                    continue

                pa_j = (E_cand[j, ] == 1)
                if np.array(est_curr["alpha"][j, pa_j]).size == 0 and np.array(est_curr["beta"][j, pa_j]).size == 0:
                    start_j = [est_curr["delta"][j], est_curr["gamma"][j],
                                np.log(est_curr["lambda"][j])]
                else:
                    start_j = [(est_curr["alpha"][j, pa_j]), est_curr["delta"][j], est_curr["beta"][j, pa_j], est_curr["gamma"][j],
                                np.log(est_curr["lambda"][j])]

                start_j = initialpars(start_j)
                pars = np.array(start_j)

                out_j = minimize(fun= lambda z : dZIHP_cpp(z, dat[:, j], dat[:, pa_j], -np.finfo(float).max, np.finfo(float).max)
                     , x0=pars, jac=lambda z: gradZIHP_cpp(z, dat[:, j], dat[:, pa_j]), method='BFGS')

                p_j = sum(pa_j)
                bic_cand = bic_curr
                bic_cand[j] = -2 * out_j.fun + (2 * p_j + 3) * np.log(n)

                if np.isfinite(np.sum(bic_cand)) and np.sum(bic_cand) < np.sum(bic_iter) - tol:
                    IMPROV = True
                    bic_iter = bic_cand
                    est_iter = est_curr.copy()
                    est_iter["E"] = E_cand
                    est_iter["alpha"][j, pa_j] = out_j.x[:p_j]
                    est_iter["beta"][j, pa_j] = out_j.x[(p_j + 1) : (2 * p_j + 1)]
                    est_iter["delta"][j] = out_j.x[p_j]
                    est_iter["gamma"][j] = out_j.x[2 * p_j + 1]
                    est_iter["lambda"][j] = np.exp(out_j.x[2 * p_j + 2])
            else:
                E_cand = est_curr["E"].copy()
                E_cand[j, k] = 1

                pa_j = (E_cand[j, ] == 1)
                if np.array(est_curr["alpha"][j, pa_j]).size == 0 and np.array(est_curr["beta"][j, pa_j]).size == 0:
                    start_j = [est_curr["delta"][j], est_curr["gamma"][j],
                                np.log(est_curr["lambda"][j])]
                else:
                    start_j = [(est_curr["alpha"][j, pa_j]), est_curr["delta"][j], est_curr["beta"][j, pa_j], est_curr["gamma"][j],
                                np.log(est_curr["lambda"][j])]

                start_j = initialpars(start_j)
                pars = np.array(start_j)

                out_j = minimize(fun= lambda z : dZIHP_cpp(z, dat[:, j], dat[:, pa_j], -np.finfo(float).max, np.finfo(float).max)
                     , x0=pars, jac=lambda z: gradZIHP_cpp(z, dat[:, j], dat[:, pa_j]), method='BFGS')

                p_j = sum(pa_j)
                bic_cand = bic_curr
                bic_cand[j] = -2 * out_j.fun + (2 * p_j + 3) * np.log(n)

                if np.isfinite(np.sum(bic_cand)) and np.sum(bic_cand) < np.sum(bic_iter) - tol:
                    IMPROV = True
                    bic_iter = bic_cand
                    est_iter = est_curr.copy()
                    est_iter["E"] = E_cand
                    est_iter["alpha"][j, pa_j] = out_j.x[:p_j]
                    est_iter["beta"][j, pa_j] = out_j.x[(p_j + 1) : (2 * p_j + 1)]
                    est_iter["delta"][j] = out_j.x[p_j]
                    est_iter["gamma"][j] = out_j.x[2 * p_j + 1]
                    est_iter["lambda"][j] = np.exp(out_j.x[2 * p_j + 2])


    id_edge = np.argwhere(est_curr['E'] == 1)
    n_rev = id_edge.shape[0]

    if n_rev > 0:
        for l in range(n_rev):
            j = id_edge[l, 0]
            k = id_edge[l, 1]
            E_cand = est_curr['E'].copy()
            E_cand[j, k] = 0
            E_cand[k, j] = 1

            # create dag for E_cand
            G_cand = create_dag(E_cand)
            if not nx.is_directed_acyclic_graph(G_cand): 
                continue

            pa_j = (E_cand[j, ] == 1)
            if np.array(est_curr["alpha"][j, pa_j]).size == 0 and np.array(est_curr["beta"][j, pa_j]).size == 0:
                start_j = [est_curr["delta"][j], est_curr["gamma"][j],
                            np.log(est_curr["lambda"][j])]
            else:
                start_j = [(est_curr["alpha"][j, pa_j]), est_curr["delta"][j], est_curr["beta"][j, pa_j], est_curr["gamma"][j],
                            np.log(est_curr["lambda"][j])]

            start_j = initialpars(start_j)
            pars = np.array(start_j)

            out_j = minimize(fun= lambda z : dZIHP_cpp(z, dat[:, j], dat[:, pa_j], -np.finfo(float).max, np.finfo(float).max)
                 , x0=pars, jac=lambda z: gradZIHP_cpp(z, dat[:, j], dat[:, pa_j]), method='BFGS')

            pa_k = (E_cand[k, ] == 1)
            if np.array(est_curr["alpha"][j, pa_k]).size == 0 and np.array(est_curr["beta"][j, pa_k]).size == 0:
                start_k = [est_curr["delta"][j], est_curr["gamma"][j],
                            np.log(est_curr["lambda"][j])]
            else:
                start_k = [(est_curr["alpha"][j, pa_k]), est_curr["delta"][j], est_curr["beta"][j, pa_k], est_curr["gamma"][j],
                            np.log(est_curr["lambda"][j])]

            start_k = initialpars(start_k)
            parsk = np.array(start_k)

            out_k = minimize(fun= lambda z : dZIHP_cpp(z, dat[:, j], dat[:, pa_k], -np.finfo(float).max, np.finfo(float).max)
                 , x0=parsk, jac=lambda z: gradZIHP_cpp(z, dat[:, j], dat[:, pa_k]), method='BFGS')

            p_j = np.sum(pa_j)
            p_k = np.sum(pa_k)
            bic_cand = bic_curr.copy()
            bic_cand[j] = -2 * out_j.fun + (2 * p_j + 3) * np.log(n)
            bic_cand[k] = -2 * out_k.fun + (2 * p_k + 3) * np.log(n)

            if np.isfinite(np.sum(bic_cand)) and np.sum(bic_cand) < np.sum(bic_iter) - tol:

                IMPROV = True
                bic_iter = bic_cand
                est_iter = est_curr.copy()
                est_iter["E"] = E_cand
                est_iter["alpha"][j, k] = 0
                est_iter["beta"][j, k] = 0
                if p_j > 0:
                    est_iter["alpha"][j, pa_j] = out_j.x[:p_j]
                    est_iter["beta"][j, pa_j] = out_j.x[(p_j + 1) : (2 * p_j + 1)]

                est_iter["delta"][j] = out_j.x[p_j]
                est_iter["gamma"][j] = out_j.x[2 * p_j + 1]
                est_iter["lambda"][j] = np.exp(out_j.x[2 * p_j + 2])

                est_iter["alpha"][j, pa_k] = out_j.x[:p_k]
                est_iter["beta"][j, pa_k] = out_j.x[(p_k + 1) : (2 * p_k + 1)]   
                est_iter["delta"][j] = out_j.x[p_k]
                est_iter["gamma"][j] = out_j.x[2 * p_k + 1]
                est_iter["lambda"][j] = np.exp(out_j.x[2 * p_k + 2])
    if (verbose):
        print("iter =", iter, "; BIC =", np.round(np.sum(bic_iter), 4), "\n")
        
    if (not IMPROV):
        break
    
    bic_curr = bic_iter.copy()
    est_curr = est_iter.copy()

out = {"est": est_curr, "bic": bic_curr, "iter": Iter}


/var/folders/2_/x3tlt3rs6j1ftvcyhmfh124c0000gn/T/ipykernel_15069/426753197.py:21: RuntimeWarning: divide by zero encountered in log
  llik = np.sum(np.log(Pi[Y0] + (1-Pi[Y0])/eval_F11[Y0]))+np.sum(np.log(1-Pi[Y1])-log_ascfacto(Lambda, y[Y1])-np.log(eval_F11[Y1]) + (y[Y1].reshape(-1,1) % np.log(Mu[Y1])))
/var/folders/2_/x3tlt3rs6j1ftvcyhmfh124c0000gn/T/ipykernel_15069/426753197.py:13: RuntimeWarning: overflow encountered in exp
  Lambda = np.exp(z[:,-1])
/var/folders/2_/x3tlt3rs6j1ftvcyhmfh124c0000gn/T/ipykernel_15069/4254713221.py:141: RuntimeWarning: overflow encountered in exp
  Lambda = np.exp(z[:,-1])
/var/folders/2_/x3tlt3rs6j1ftvcyhmfh124c0000gn/T/ipykernel_15069/4254713221.py:94: RuntimeWarning: invalid value encountered in true_divide
  fac *= (digamma(b+1)/digamma(b)) * (z/b)


KeyboardInterrupt: 

In [813]:
est_curr['E']

array([[0., 0., 0., 0., 0.],
       [0., 0., 0., 0., 0.],
       [0., 0., 0., 0., 0.],
       [0., 0., 0., 0., 0.],
       [0., 0., 0., 0., 0.]])

In [1050]:
hc_linear_zihp(dat, starting_dag = None, maxiter = 500, tol = 1e-12, verbose = True)

/var/folders/2_/x3tlt3rs6j1ftvcyhmfh124c0000gn/T/ipykernel_15069/3953531946.py:13: RuntimeWarning: overflow encountered in exp
  Lambda = np.exp(z[:,-1])
/var/folders/2_/x3tlt3rs6j1ftvcyhmfh124c0000gn/T/ipykernel_15069/3953531946.py:21: RuntimeWarning: divide by zero encountered in log
  llik = np.sum(np.log(Pi[Y0] + (1-Pi[Y0])/eval_F11[Y0]))+np.sum(np.log(1-Pi[Y1])-log_ascfacto(Lambda, y[Y1])-np.log(eval_F11[Y1]) + (y[Y1].reshape(-1,1) % np.log(Mu[Y1])))
/var/folders/2_/x3tlt3rs6j1ftvcyhmfh124c0000gn/T/ipykernel_15069/4254713221.py:141: RuntimeWarning: overflow encountered in exp
  Lambda = np.exp(z[:,-1])
/var/folders/2_/x3tlt3rs6j1ftvcyhmfh124c0000gn/T/ipykernel_15069/4254713221.py:94: RuntimeWarning: invalid value encountered in true_divide
  fac *= (digamma(b+1)/digamma(b)) * (z/b)


KeyboardInterrupt: 

In [832]:
def hc_linear_zihp(dat, starting_dag = None, maxiter=500, tol=1e-12, optim_control=None, verbose=False):
    n, p = dat.shape

    if optim_control is None:
        optim_control = {"fnscale": -1, "maxiter": 10000, "reltol": 1.0e-8}

    if starting_dag is None:
        starting_dag = np.zeros((p, p))
        
    bic_curr = np.empty(p)
    est_curr = {
        "E": starting_dag,
        "alpha": np.zeros((p, p)),
        "beta": np.zeros((p, p)),
        "delta": np.zeros(p),
        "gamma": np.zeros(p),
        "lambda": np.ones(p),
    }

    for j in range(p):

        pa_j = (est_curr['E'][j, :] == 1)

        if np.array(est_curr["alpha"][j, pa_j]).size == 0 and np.array(est_curr["beta"][j, pa_j]).size == 0:
            start_j = [est_curr["delta"][j], est_curr["gamma"][j],
                        np.log(est_curr["lambda"][j])]
        else:
            start_j = [np.array(est_curr["alpha"][j, pa_j]), est_curr["delta"][j], est_curr["beta"][j, pa_j], est_curr["gamma"][j],
                        np.log(est_curr["lambda"][j])]

        pars = np.array(start_j, dtype=object)

        out_j = minimize(fun= lambda z : dZIHP_cpp(z, dat[:, 0], dat[:, pa_j], -np.finfo(float).max, np.finfo(float).max)
             , x0=pars, jac=lambda z: gradZIHP_cpp(z, dat[:, j], dat[:, pa_j]), method='BFGS')

        p_j = sum(pa_j)
        bic_curr[j] = -2 * out_j.fun + (2 * p_j + 3) * np.log(n)

        if p_j > 0:

            est_curr["alpha"][j, pa_j] = out_j.x[0 : p_j]
            est_curr["beta"][j, pa_j]  = out_j.x[(p_j + 1) : (2 * p_j + 1)]

        est_curr["delta"][j]  = out_j.x[p_j]
        est_curr["gamma"][j]  = out_j.x[2 * p_j + 1]
        est_curr["lambda"][j] = np.exp(out_j.x[2 * p_j + 2])
    
    # start hill climbing
    
    bic_iter = bic_curr
    est_iter = est_curr

    for Iter in range(maxiter):
        IMPROV = False
        for j in range(p):
            for k in range(p):
                if j == k:
                    continue

                if est_curr["E"][j, k] == 0:
                    E_cand = est_curr["E"].copy()
                    E_cand[j, k] = 1

                    # create dag for E_cand
                    G_cand = create_dag(E_cand)
                    if not nx.is_directed_acyclic_graph(G_cand): 
                        continue

                    pa_j = (E_cand[j, ] == 1)
                    if np.array(est_curr["alpha"][j, pa_j]).size == 0 and np.array(est_curr["beta"][j, pa_j]).size == 0:
                        start_j = [est_curr["delta"][j], est_curr["gamma"][j],
                                    np.log(est_curr["lambda"][j])]
                    else:
                        start_j = [np.array(est_curr["alpha"][j, pa_j]), est_curr["delta"][j], est_curr["beta"][j, pa_j], est_curr["gamma"][j],
                                    np.log(est_curr["lambda"][j])]
                    pars = np.array(start_j, dtype=object)

                    out_j = minimize(fun= lambda z : dZIHP_cpp(z, dat[:, j], dat[:, pa_j], -np.finfo(float).max, np.finfo(float).max)
                         , x0=pars, jac=lambda z: gradZIHP_cpp(z, dat[:, j], dat[:, pa_j]), method='BFGS')

                    p_j = sum(pa_j)
                    bic_cand = bic_curr
                    bic_cand[j] = -2 * out_j.fun + (2 * p_j + 3) * np.log(n)

                    if np.isfinite(np.sum(bic_cand)) and np.sum(bic_cand) < np.sum(bic_iter) - tol:
                        IMPROV = True
                        bic_iter = bic_cand
                        est_iter = est_curr.copy()
                        est_iter["E"] = E_cand
                        est_iter["alpha"][j, pa_j] = out_j.x[:p_j]
                        est_iter["beta"][j, pa_j] = out_j.x[(p_j + 1) : (2 * p_j + 1)]
                        est_iter["delta"][j] = out_j.x[p_j]
                        est_iter["gamma"][j] = out_j.x[2 * p_j + 1]
                        est_iter["lambda"][j] = np.exp(out_j.x[2 * p_j + 2])
                else:
                    E_cand = est_curr["E"].copy()
                    E_cand[j, k] = 1

                    pa_j = (E_cand[j, ] == 1)
                    if np.array(est_curr["alpha"][j, pa_j]).size == 0 and np.array(est_curr["beta"][j, pa_j]).size == 0:
                        start_j = [est_curr["delta"][j], est_curr["gamma"][j],
                                    np.log(est_curr["lambda"][j])]
                    else:
                        start_j = [np.array(est_curr["alpha"][j, pa_j]), est_curr["delta"][j], est_curr["beta"][j, pa_j], est_curr["gamma"][j],
                                    np.log(est_curr["lambda"][j])]
                    pars = np.array(start_j, dtype=object)

                    out_j = minimize(fun= lambda z : dZIHP_cpp(z, dat[:, j], dat[:, pa_j], -np.finfo(float).max, np.finfo(float).max)
                         , x0=pars, jac=lambda z: gradZIHP_cpp(z, dat[:, j], dat[:, pa_j]), method='BFGS')

                    p_j = sum(pa_j)
                    bic_cand = bic_curr
                    bic_cand[j] = -2 * out_j.fun + (2 * p_j + 3) * np.log(n)

                    if np.isfinite(np.sum(bic_cand)) and np.sum(bic_cand) < np.sum(bic_iter) - tol:
                        IMPROV = True
                        bic_iter = bic_cand
                        est_iter = est_curr.copy()
                        est_iter["E"] = E_cand
                        est_iter["alpha"][j, pa_j] = out_j.x[:p_j]
                        est_iter["beta"][j, pa_j] = out_j.x[(p_j + 1) : (2 * p_j + 1)]
                        est_iter["delta"][j] = out_j.x[p_j]
                        est_iter["gamma"][j] = out_j.x[2 * p_j + 1]
                        est_iter["lambda"][j] = np.exp(out_j.x[2 * p_j + 2])


        id_edge = np.argwhere(est_curr['E'] == 1)
        n_rev = id_edge.shape[0]

        if n_rev > 0:
            for l in range(n_rev):
                j = id_edge[l, 0]
                k = id_edge[l, 1]
                E_cand = est_curr['E'].copy()
                E_cand[j, k] = 0
                E_cand[k, j] = 1

                # create dag for E_cand
                G_cand = create_dag(E_cand)
                if not nx.is_directed_acyclic_graph(G_cand): 
                    continue

                pa_j = (E_cand[j, ] == 1)
                if np.array(est_curr["alpha"][j, pa_j]).size == 0 and np.array(est_curr["beta"][j, pa_j]).size == 0:
                    start_j = [est_curr["delta"][j], est_curr["gamma"][j],
                                np.log(est_curr["lambda"][j])]
                else:
                    start_j = [np.array(est_curr["alpha"][j, pa_j]), est_curr["delta"][j], est_curr["beta"][j, pa_j], est_curr["gamma"][j],
                                np.log(est_curr["lambda"][j])]
                pars = np.array(start_j, dtype=object)

                out_j = minimize(fun= lambda z : dZIHP_cpp(z, dat[:, j], dat[:, pa_j], -np.finfo(float).max, np.finfo(float).max)
                     , x0=pars, jac=lambda z: gradZIHP_cpp(z, dat[:, j], dat[:, pa_j]), method='BFGS')

                pa_k = (E_cand[k, ] == 1)
                if np.array(est_curr["alpha"][j, pa_k]).size == 0 and np.array(est_curr["beta"][j, pa_k]).size == 0:
                    start_j = [est_curr["delta"][j], est_curr["gamma"][j],
                                np.log(est_curr["lambda"][j])]
                else:
                    start_j = [np.array(est_curr["alpha"][j, pa_k]), est_curr["delta"][j], est_curr["beta"][j, pa_k], est_curr["gamma"][j],
                                np.log(est_curr["lambda"][j])]
                parsk = np.array(start_j, dtype=object)

                out_k = minimize(fun= lambda z : dZIHP_cpp(z, dat[:, j], dat[:, pa_k], -np.finfo(float).max, np.finfo(float).max)
                     , x0=parsk, jac=lambda z: gradZIHP_cpp(z, dat[:, j], dat[:, pa_k]), method='BFGS')

                p_j = np.sum(pa_j)
                p_k = np.sum(pa_k)
                bic_cand = bic_curr.copy()
                bic_cand[j] = -2 * out_j.fun + (2 * p_j + 3) * np.log(n)
                bic_cand[k] = -2 * out_k.fun + (2 * p_k + 3) * np.log(n)

                if np.isfinite(np.sum(bic_cand)) and np.sum(bic_cand) < np.sum(bic_iter) - tol:
                    IMPROV = True
                    bic_iter = bic_cand
                    est_iter = est_curr.copy()
                    est_iter["E"] = E_cand
                    est_iter["alpha"][j, k] = 0
                    est_iter["beta"][j, k] = 0
                    if p_j > 0:
                        est_iter["alpha"][j, pa_j] = out_j.x[:p_j]
                        est_iter["beta"][j, pa_j] = out_j.x[(p_j + 1) : (2 * p_j + 1)]

                    est_iter["delta"][j] = out_j.x[p_j]
                    est_iter["gamma"][j] = out_j.x[2 * p_j + 1]
                    est_iter["lambda"][j] = np.exp(out_j.x[2 * p_j + 2])

                    est_iter["alpha"][j, pa_k] = out_j.x[:p_k]
                    est_iter["beta"][j, pa_k] = out_j.x[(p_k + 1) : (2 * p_k + 1)]   
                    est_iter["delta"][j] = out_j.x[p_k]
                    est_iter["gamma"][j] = out_j.x[2 * p_k + 1]
                    est_iter["lambda"][j] = np.exp(out_j.x[2 * p_k + 2])
                    
        if (verbose):
            print("iter =", Iter, "; BIC =", np.round(np.sum(bic_iter), 4), "\n")
        
        print(IMPROV)
        if (not IMPROV):
            break
        
        bic_curr = bic_iter.copy()
        est_curr = est_iter.copy()
    
    out = {"est": est_curr, "bic": bic_curr, "iter": Iter}
    
    return out

In [ ]:
'''
workable function for calculating likelihood
'''

x = dat[: ,pa_j]
y = dat[:, j]
z = pars
z = z.reshape(1,-1)

pn, pp = x.shape
x1 = np.concatenate([x, np.ones((n,1))], axis = 1)

Pi = np.exp(x1 @ z[:,0:pp+1].T)
Pi = Pi/(1+Pi)
Pi[~np.isfinite(Pi)] = 1

Mu = np.exp(x1 @ z[:,pp+1:2*pp+2].T)
Lambda = np.exp(z[:,-1])

Y0 = np.where(y<=1e-8)
Y1 = np.where(y>1e-8)

mod = np.mod(y[Y1].reshape(-1,1),np.log(Mu[Y1]).reshape(-1,1))
mod[np.isnan(mod)] = 0

llik = np.sum(np.log(Pi[Y0] + (1-Pi[Y0])/1))+np.sum(np.log(1-Pi[Y1])-0-0 + mod)

# workable minimize function for dat[:, pa_j] is none.
'''
minimize(fun= lambda z : dZIHP_cpp(z, dat[:, 0], dat[:, pa_j], -np.finfo(float).max, np.finfo(float).max)
         , x0=pars, jac=lambda z: gradZIHP_cpp(z, dat[:, j], dat[:, pa_j]), method='BFGS')
'''